<a href="https://colab.research.google.com/github/MILAGROSCHALBAUD/TP-ANALISIS-PREDICITIVO/blob/main/FINAL_ANALISIS_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#instalo paquetes que necesito
install.packages("rpart")
install.packages("corrplot")
install.packages("caret")
install.packages("fastDummies")
install.packages("dplyr")
install.packages("Hmisc")
install.packages("rpart.plot")
install.packages("ggplot2")
install.packages("funModeling")

In [ ]:
library(dplyr)
library(funModeling)
library(ggplot2)
library(tidyverse)

In [ ]:
#leo bases de datos
datos <- read.csv("datos.csv",header=TRUE,sep = ",")
resultados <-read.csv("resultados.csv",header=TRUE,sep=",")

In [ ]:
#veo bases de datos
str(datos)
str(resultados)

In [ ]:
#como calculo el promedio de numVotes para establacer como filtro
#prom=sum(datos$numVotes)/
#missings totales
#library(mice)
#md.pattern(datos, rotate.names=90)
#md.pattern(datos,plot=TRUE, rotate.names=FALSE)

In [ ]:
ggplot(data = datos, aes(x = titleType, y = averageRating, colour = titleType)) +
  geom_boxplot() + coord_flip()


In [ ]:
ggplot(data = datos, aes(x = directors, y = averageRating, colour = directors)) +
  geom_boxplot() + coord_flip()


In [ ]:
ggplot(data = datos, aes(x = genres_x, y = averageRating, colour = genres_x)) +
  geom_boxplot() + coord_flip()

In [ ]:
ggplot(data = datos, aes(x = startYear, y = averageRating, colour = startYear)) +
  geom_jitter() + coord_flip()

In [ ]:
ggplot(data = datos, aes(x = endYear, y = averageRating, colour = endYear)) +
  geom_jitter() + coord_flip()

In [ ]:
#analizo missings de numericas
mean(is.na(datos$averageRating))
mean(is.na(datos$popularity))
mean(is.na(datos$runtime))
mean(is.na(datos$budget))
mean(is.na(datos$revenue))
mean(is.na(datos$isOriginalTitle))
mean(is.na(datos$ordering))
mean(is.na(datos$numVotes))

In [ ]:
#completo los NA con ceros para aquellas variables con mas del 63 % de missings (no las uso en el modelo)
datos$popularity[is.na(datos$popularity)]=0
datos$runtime[is.na(datos$runtime)]=0
datos$budget[is.na(datos$budget)]=0
datos$revenue[is.na(datos$revenue)]=0


In [ ]:
#completo los NA con la media para aquellas variables con menos del 63% de missings
datos$isOriginalTitle <- ifelse(is.na(datos$isOriginalTitle),1,datos$isOriginalTitle)
datos$ordering[is.na(datos$ordering)]=mean(datos$ordering,na.rm=TRUE)


In [ ]:
#trabajo con isAdult
datos$isAdult[datos$isAdult == "2020"] <- "0"
datos$isAdult <- ifelse(is.na(datos$isAdult),0,datos$isAdult)
datos$isAdult <- as.factor(datos$isAdult)
str(datos$isAdult)

 Factor w/ 2 levels "0","1": 1 1 1 1 1 1 1 1 1 1 ...


In [ ]:
#compruebo que se hayan cambiado lo NA.... esto lo puedo borrar
mean(is.na(datos$popularity))
mean(is.na(datos$runtime))
mean(is.na(datos$budget))
mean(is.na(datos$revenue))
mean(is.na(datos$isOriginalTitle))
mean(is.na(datos$ordering))
mean(is.na(datos$numvotes))

In [ ]:
#trabajo con seasonNumber
#table(datos$seasonNumber)
datos$seasonNumber[datos$seasonNumber == ""] <- NA
datos$seasonNumber=as.numeric(datos$seasonNumber)
datos$seasonNumber[is.na(datos$seasonNumber)]=median(datos$seasonNumber,na.rm=TRUE)
str(datos$seasonNumber)

In [ ]:
#datos$seasonNumber=as.numeric(datos$seasonNumber)
#datos$seasonNumber[is.na(datos$seasonNumber)]=median(datos$seasonNumber,na.rm=TRUE)
#str(datos$seasonNumber)

In [ ]:
mean(is.na(datos$seasonNumber))

[1] 0

In [ ]:
#trabajo con episodeNumber
#table(datos$episodeNumber)
datos$episodeNumber[datos$episodeNumber == ""] <- NA
datos$episodeNumber<-as.numeric(datos$episodeNumber)
datos$episodeNumber[is.na(datos$episodeNumber)]= median(datos$episodeNumber,na.rm=TRUE)
str(datos$episodeNumber)
#datos$episodeNumber<-as.numeric(datos$episodeNumber)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


 num [1:900000] 5 9 8 8 9 12 25 8 8 8 ...


In [ ]:
mean(is.na(datos$episodeNumber))

[1] 0

In [ ]:
#datos$episodeNumber<-as.numeric(datos$episodeNumber)
#datos$episodeNumber[is.na(datos$episodeNumber)]= median(datos$episodeNumber,na.rm=TRUE)
#str(datos$episodeNumber)

In [ ]:
#trabajo con startYear 
#table(datos$startYear)
datos$startYear[datos$startYear == "\\N"] <- NA
datos$startYear[datos$startYear ==" "]<- NA
datos$startYear<-as.numeric(datos$startYear)
datos$startYear[is.na(datos$startYear)]=median(datos$startYear,na.rm=TRUE)
str(datos$startYear)

 num [1:900000] 2014 2020 2006 2016 2014 ...


In [ ]:
mean(is.na(datos$startYear))

[1] 0

In [ ]:
#datos$startYear<-as.numeric(datos$startYear)
#datos$startYear[is.na(datos$starYear)]=median(datos$startYear,na.rm=TRUE)
#str(datos$startYear)

 num [1:900000] 2014 2020 2006 2016 2014 ...


In [ ]:
#trabajo con endYear
#table(datos$endYear)
datos$endYear[datos$endYear == "\\N"] <- NA
datos$endYear[datos$endYear == " "] <- NA
datos$endYear<-as.numeric(datos$endYear)
datos$endYear <- ifelse(is.na(datos$endYear),2021,datos$endYear)

In [ ]:
#datos$endYear<-as.numeric(datos$endYear)
#datos$endYear <- ifelse(is.na(datos$endYear),2021,datos$endYear)

In [ ]:
mean(is.na(datos$endYear))

[1] 0

In [ ]:
#trabajo con runtimeMinutes
#table(datos$runtimeMinutes)
datos$runtimeMinutes[datos$runtimeMinutes == "\\N"] <- NA

In [ ]:
datos$runtimeMinutes=as.numeric(datos$runtimeMinutes)
datos$runtimeMinutes[is.na(datos$runtimeMinutes)]=median(datos$runtimeMinutes,na.rm=TRUE)
str(datos$runtimeMinutes)

In [ ]:
mean(is.na(datos$runtimeMinutes))

In [ ]:
#trabajo con title type
datos$titleType[datos$titleType == ""] <- "tvEpisode"
datos$titleType <- as.factor(datos$titleType)

datos %>% group_by(titleType) %>%
    summarise(rating=mean(averageRating),count=n()) %>% 
    arrange(desc(rating))


In [ ]:
datos$tvEpisode <- ifelse(datos$titleType=='tvEpisode',1,0)
datos$movie <- ifelse(datos$titleType=='movie',1,0)
datos$video <- ifelse(datos$titleType=='video',1,0)
datos$tvMovie <- ifelse(datos$titleType=='tvMovie',1,0)
datos$short <- ifelse(datos$titleType=='short',1,0)
datos$tvSeries <- ifelse(datos$titleType=='tvSeries',1,0)
datos$tvShort <- ifelse(datos$titleType=="tvShort",1,0)
datos$tvMiniSeries <- ifelse(datos$titleType=="tvMiniSeries",1,0)
datos$tvSpecial <- ifelse(datos$titleType=="tvSpecial",1,0)
datos$videogame <- ifelse(datos$titleType=="videoGame",1,0)

In [ ]:
#trabajo con generos_x y separo en top y losers 
datos %>%
    group_by(genres_x) %>%
    summarise(cantidad=n(),avg_score=mean(averageRating)) %>%
    filter(cantidad>10) %>% 
    arrange(desc(avg_score))

gt <- datos %>% group_by(genres_x) %>%
    summarise(avg_score=mean(averageRating),cantidad=n(),puntos=sum(numVotes)) %>%
    arrange(desc(avg_score)) %>% 
    filter(puntos >4000, avg_score > 8)

gl <- datos %>% group_by(genres_x) %>%
    summarise(avg_score=mean(averageRating),cantidad=n(),puntos=sum(numVotes)) %>%
    arrange(desc(avg_score)) %>% 
    filter(puntos > 4000, avg_score<= 4)

nrow(gt)
nrow(gl)

In [ ]:
gt

In [ ]:
gl

In [ ]:
#datos$writers[datos$writers == "\\N"] <- "nm1974030"	

In [ ]:
#trabajo con writers y los divido en top y losers
datos %>%
    group_by(writers) %>%
    summarise(cantidad=n(),avg_score=mean(averageRating)) %>%
    filter(cantidad>10) %>% 
    arrange(desc(avg_score))

wt <- datos %>% group_by(writers) %>%
    summarise(avg_score=mean(averageRating),cantidad=n(), puntos=sum(numVotes)) %>%
    arrange(desc(avg_score)) %>% 
    filter(puntos > 4000, avg_score>8)

wl <- datos %>% group_by(writers) %>%
    summarise(avg_score=mean(averageRating),cantidad=n(), puntos=sum(numVotes)) %>%
    arrange(desc(avg_score)) %>% 
    filter(puntos > 4000, avg_score<=4)

nrow(wt)
nrow(wl)

In [ ]:
#completo los NA de directors
#datos$directors[datos$directors== "\\N"] <- "nm1475126"

In [ ]:
# trabajo con directors y los divido en top y losers
datos %>%
    group_by(directors) %>%
    summarise(cantidad=n(),avg_score=mean(averageRating)) %>%
    filter (cantidad > 10) %>% 
    arrange(desc(avg_score))

dt <- datos %>% group_by(directors) %>%
    summarise(avg_score=mean(averageRating),cantidad=n(), puntos=sum(numVotes)) %>%
    arrange(desc(avg_score)) %>% 
    filter(puntos >4000, avg_score > 8)

dl <- datos %>% group_by(directors) %>%
    summarise(avg_score=mean(averageRating),cantidad=n(), puntos=sum(numVotes)) %>%
    arrange(desc(avg_score)) %>% 
    filter(puntos > 4000, avg_score<=4)

nrow(dt)
nrow(dl)

In [ ]:
datos %>%
    group_by(production_companies) %>%
    summarise(cantidad=n(),avg_score=mean(averageRating)) %>%
    filter(cantidad>10) %>% 
    arrange(desc(avg_score))

prt <- datos %>% group_by(production_companies) %>%
    summarise(avg_score=mean(averageRating),cantidad=n(), puntos=sum(numVotes)) %>%
    arrange(desc(avg_score)) %>% 
    filter(puntos > 4000, avg_score>8)

prl <- datos %>% group_by(production_companies) %>%
    summarise(avg_score=mean(averageRating),cantidad=n(), puntos=sum(numVotes)) %>%
    arrange(desc(avg_score)) %>% 
    filter(puntos > 4000, avg_score<=4)

nrow(prt)
nrow(prl)

In [ ]:
datos %>%
    group_by(production_countries) %>%
    summarise(cantidad=n(),avg_score=mean(averageRating)) %>%
    filter(cantidad>10) %>% 
    arrange(desc(avg_score))

ct <- datos %>% group_by(production_countries) %>%
    summarise(avg_score=mean(averageRating),cantidad=n(), puntos=sum(numVotes)) %>%
    arrange(desc(avg_score)) %>% 
    filter(puntos > 4000, avg_score>8)

cl <- datos %>% group_by(production_countries) %>%
    summarise(avg_score=mean(averageRating),cantidad=n(), puntos=sum(numVotes)) %>%
    arrange(desc(avg_score)) %>% 
    filter(puntos > 4000, avg_score<=4)

nrow(ct)
nrow(cl)

In [ ]:
#duracion total de los proyectos
datos$duracion = (datos$endYear)-(datos$startYear)
datos$duracion <-as.numeric(datos$duracion)

In [ ]:
datos$directortop <- ifelse(datos$directors %in% dt$directors,1,0)
datos$directorloser <- ifelse(datos$directors %in% dl$directors,1,0)
datos$writertop <- ifelse(datos$writers %in% wt$writers,1,0)
datos$writerloser <- ifelse(datos$writers %in% wl$writers,1,0)
datos$generotop <- ifelse(datos$genres_x %in% gt$genres_x,1,0)
datos$generoloser <- ifelse(datos$genres_x %in% gl$genres_x,1,0)
datos$companietop <- ifelse(datos$production_companies %in% prt$production_companies,1,0)
datos$companieloser <- ifelse(datos$production_companies %in% prl$production_companies,1,0)
datos$paistop <- ifelse(datos$production_countries %in% ct$production_countries,1,0)
datos$paisloser <-ifelse(datos$production_countries %in% cl$production_countries,1,0)
sum(datos$directortop)
sum(datos$directorloser)
sum(datos$writertop)
sum(datos$writerloser)
sum(datos$generotop)
sum(datos$generoloser)
sum(datos$paistop)
sum(datos$paisloser)
sum(datos$companieloser)
sum(datos$companietop)

In [ ]:
#datos$startYear=NULL
datos$language=NULL
datos$runtime=NULL
datos$budget=NULL
datos$popularity=NULL
datos$revenue=NULL
#datos$isOriginalTitle=NULL
datos$overview=NULL
datos$production_companies=NULL
datos$production_countries=NULL
datos$release_date=NULL
datos$genres_y=NULL
datos$original_language=NULL
datos$attributes=NULL
#datos$runtimeMinutes=NULL 
datos$status=NULL
datos$video=NULL
datos$tagline=NULL
datos$adult=NULL
datos$types=NULL
#datos$ordering=NULL
datos$endYear=NULL
#datos$directors=NULL
#datos$writers=NULL
#datos$seasonNumber=NULL
#datos$episodeNumber=NULL
#datos$isAdult=NULL
#datos$genres_x=NULL

In [ ]:
d=str(datos)

In [ ]:
write.csv(d,"BASE_DATOS1.csv")

In [ ]:
#limpieza de base resultados
resultados$popularity[is.na(resultados$popularity)]=0
resultados$runtime[is.na(resultados$runtime)]=0
resultados$budget[is.na(resultados$budget)]=0
resultados$revenue[is.na(resultados$revenue)]=0

In [ ]:
table(resultados$runtimeMinutes)
resultados$runtimeMinutes[resultados$runtimeMinutes == ""] <- "\\N"
resultados$runtimeMinutes=as.numeric(resultados$runtimeMinutes)
resultados$runtimeMinutes[is.na(resultados$runtimeMinutes)]=median(resultados$runtimeMinutes,na.rm=TRUE)

In [ ]:
mean(is.na(resultados$runtimeMinutes))

In [ ]:
#table(resultados$startYear)
resultados$startYear[resultados$startYear == "\\N"] <- NA
resultados$startYear[resultados$startYear == " "] <- NA
resultados$startYear<-as.numeric(resultados$startYear)
resultados$startYear[is.na(resultados$startYear)]=median(resultados$startYear,na.rm=TRUE)


In [ ]:
#resultados$startYear<-as.numeric(resultados$startYear)

In [ ]:
mean(is.na(resultados$startYear))

[1] 0

In [ ]:
resultados$isOriginalTitle <- ifelse(is.na(resultados$isOriginalTitle),1,resultados$isOriginalTitle)
str(resultados$isOriginalTitle)
resultados$ordering[is.na(resultados$ordering)]=mean(resultados$ordering,na.rm=TRUE)
str(resultados$ordering)

In [ ]:
resultados$isAdult[resultados$isAdult == "2014"] <- "0"
resultados$isAdult <- ifelse(is.na(resultados$isAdult),0,resultados$isAdult)
resultados$isAdult <- as.factor(resultados$isAdult)
str(resultados$isAdult)


In [ ]:
#trabajo con endYear
#table(resultados$endYear)
resultados$endYear[resultados$endYear == "\\N"] <- NA
resultados$endYear[resultados$endYear == " "] <- NA
resultados$endYear<-as.numeric(resultados$endYear)
resultados$endYear <- ifelse(is.na(resultados$endYear),2021,resultados$endYear)


In [ ]:
mean(is.na(resultados$endYear))

[1] 0

In [ ]:
#trabajo con episodeNumber
#table(resultados$episodeNumber)
resultados$episodeNumber[resultados$episodeNumber == ""] <- NA
resultados$episodeNumber<-as.numeric(resultados$episodeNumber)
resultados$episodeNumber[is.na(resultados$episodeNumber)]= median(resultados$episodeNumber,na.rm=TRUE)

In [ ]:
mean(is.na(resultados$episodeNumber))

In [ ]:
#trabajo con seasonNumber
#table(resultados$seasonNumber)
resultados$seasonNumber[resultados$seasonNumber == ""] <- NA
resultados$seasonNumber=as.numeric(resultados$seasonNumber)
resultados$seasonNumber[is.na(resultados$seasonNumber)]=median(resultados$seasonNumber,na.rm=TRUE)
str(datos$seasonNumber)

In [ ]:
mean(is.na(resultados$seasonNumber))

In [ ]:
#completo los NA de directors
resultados$directors[resultados$directors== "\\N"] <- "nm1475126"

In [ ]:
#completo los NA de writers
resultados$writers[resultados$writers == "\\N"] <- "nm1974030"

In [ ]:
#trabajo con title type
resultados$titleType[resultados$titleType == ""] <- "tvEpisode"
resultados$titleType <- as.factor(resultados$titleType)

datos %>% group_by(titleType) %>%
    summarise(rating=mean(averageRating),count=n()) %>% 
    arrange(desc(rating))

In [ ]:
resultados$tvEpisode <- ifelse(resultados$titleType=='tvEpisode',1,0)
resultados$movie <- ifelse(resultados$titleType=='movie',1,0)
resultados$video <- ifelse(resultados$titleType=='video',1,0)
resultados$tvMovie <- ifelse(resultados$titleType=='tvMovie',1,0)
resultados$short <- ifelse(resultados$titleType=='short',1,0)
resultados$tvSeries <- ifelse(resultados$titleType=='tvSeries',1,0)
resultados$tvShort <- ifelse(resultados$titleType=="tvShorts",1,0)
resultados$tvMiniSeries <- ifelse(resultados$titleType=="tvMiniSeries",1,0)
resultados$tvSpecial <- ifelse(resultados$titleType=="tvSpecial",1,0)
resultados$videogame <- ifelse(resultados$titleType=="videogame",1,0)

In [ ]:
#directores-writers-generos
resultados$directortop <- ifelse(resultados$directors %in% dt$directors,1,0)
resultados$directorloser <- ifelse(resultados$directors %in% dl$directors,1,0)
resultados$writertop <- ifelse(resultados$writers %in% wt$writers,1,0)
resultados$writerloser <- ifelse(resultados$writers %in% wl$writers,1,0)
resultados$generotop <- ifelse(resultados$genres_x %in% gt$genres_x,1,0)
resultados$generoloser <- ifelse(resultados$genres_x %in% gl$genres_x,1,0)
resultados$companietop <- ifelse(resultados$production_companies %in% prt$production_companies,1,0)
resultados$companieloser <- ifelse(resultados$production_companies %in% prl$production_companies,1,0)
resultados$paistop <- ifelse(resultados$production_countries %in% ct$production_countries,1,0)
resultados$paisloser <-ifelse(resultados$production_countries %in% cl$production_countries,1,0)
sum(resultados$directortop)
sum(resultados$directorloser)
sum(resultados$writertop)
sum(resultados$writerloser)
sum(resultados$generotop)
sum(resultados$generoloser)
sum(resultados$companieloser)
sum(resultados$companietop)
sum(resultados$paistop)
sum(resultados$paisloser)

In [ ]:
resultados$duracion=(resultados$endYear)-(resultados$startYear)
str(resultados$duracion)

 num [1:321932] 22 14 50 20 7 69 2 11 10 11 ...


In [ ]:
#resultados$startYear=NULL
resultados$language=NULL
resultados$runtime=NULL
resultados$budget=NULL
resultados$popularity=NULL
resultados$revenue=NULL
#resultados$isOriginalTitle=NULL
resultados$overview=NULL
resultados$production_companies=NULL
resultados$production_countries=NULL
resultados$release_date=NULL
resultados$genres_y=NULL
resultados$original_language=NULL
resultados$attributes=NULL
#resultados$runtimeMinutes=NULL 
resultados$status=NULL
resultados$video=NULL
resultados$tagline=NULL
resultados$adult=NULL
resultados$types=NULL
#resultados$ordering=NULL
#resultados$endYear=NULL
#resultados$directors=NULL
#resultados$writers=NULL
#resultados$seasonNumber=NULL
#resultados$episodeNumber=NULL
#resultados$isAdult=NULL
#resultados$genres_x=NULL

In [ ]:
r=str(resultados)

In [ ]:
write.csv(r,"BASE_RESULTADOS.csv")

In [ ]:
d2=datos

In [ ]:
r2=resultados

In [ ]:
#hago la partición y me voy a python a correr los modelos r no me deja 
library(caret)
set.seed(123);particion=createDataPartition(y=d2$averageRating,p=0.75,list=FALSE)
entrenar=d2[particion,]
test=d2[-particion,]

In [ ]:
#regresion <- lm(averageRating ~., data = entrenar)
#summary(regresion)

In [ ]:
#genero base train para python
write.csv(entrenar,"ENTRENAR1.csv")

In [ ]:
#genero base test para python
write.csv(test,"TEST1.csv")

In [ ]:
#llamo base resultado limpia para python 
write.csv(r2,"RESULTADO1.csv")